In [3]:
import pickle
import gensim

In [4]:
#reading the variables from file : dictionary and tdf/corpus
with open(r"C:\Users\kanis\Major-Project\Tokens\tkn01.pkl","rb") as fp:
    data = pickle.load(fp)

dictionary = data[0]
termdocfreq = data[1]
finallemma = data[2]
feedtodict = data[3]

In [5]:
from pprint import pprint

# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=termdocfreq,
                                       id2word=dictionary,
                                       num_topics=num_topics,random_state=100, chunksize=100, passes=10, per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[termdocfreq]

[(0,
  '0.036*"word" + 0.032*"language" + 0.024*"learning" + 0.020*"learn" + '
  '0.015*"thing" + 0.014*"help" + 0.013*"one" + 0.012*"reading" + 0.011*"day" '
  '+ 0.011*"time"'),
 (1,
  '0.035*"language" + 0.034*"thing" + 0.030*"watch" + 0.030*"show" + '
  '0.029*"movie" + 0.026*"reading" + 0.024*"phrase" + 0.024*"trying" + '
  '0.023*"like" + 0.016*"Possibly"'),
 (2,
  '0.039*"word" + 0.026*"language" + 0.023*"learning" + 0.020*"learn" + '
  '0.017*"help" + 0.014*"important" + 0.012*"thing" + 0.012*"phrase" + '
  '0.011*"reading" + 0.011*"conversation"'),
 (3,
  '0.033*"language" + 0.027*"learning" + 0.025*"learn" + 0.024*"phrase" + '
  '0.020*"important" + 0.019*"word" + 0.018*"thing" + 0.018*"reading" + '
  '0.016*"understand" + 0.015*"sentence"'),
 (4,
  '0.018*"word" + 0.017*"learning" + 0.016*"language" + 0.013*"learn" + '
  '0.010*"help" + 0.009*"reading" + 0.008*"new" + 0.008*"phrase" + '
  '0.008*"grammar" + 0.008*"time"'),
 (5,
  '0.040*"language" + 0.031*"word" + 0.025*"lea

In [6]:
#coherence and score 
from gensim.models import CoherenceModel
# instantiate topic coherence model
import numpy as np
newcorpus = []
for lis in finallemma :
    for w in lis :
        if len(w)>2:
            newcorpus.append(w)

cm = CoherenceModel(model=lda_model, texts=feedtodict, dictionary=dictionary, coherence='c_v')
with np.errstate(invalid='ignore'):
    coherence_lda = cm.get_coherence()
    print('Coherence: ', coherence_lda)                
# getting topic coherence score 


#getting the perplexity
print('\nPerplexity: ', lda_model.log_perplexity(termdocfreq))

Coherence:  0.42470777805264603

Perplexity:  -5.127869182828577


In [7]:
#import pickle 
#import pyLDAvis
#import os 
#import pyLDAvis.gensim


#num_topics = 10
import pyLDAvis
import pyLDAvis.gensim 

# Visualize the topics
pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, termdocfreq, dictionary,sort_topics = 'false')
#vis
#LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))

#LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, termdocfreq, dictionary)
    

In [8]:
from nltk import tokenize

file10 = open(r"C:\Users\kanis\Major-Project\Intermediate\itm01.txt","r+")

Lines = file10.readlines()
#print(Lines)
x = ""
for line in Lines:
    x = x + line
#print(x)
tksen = tokenize.sent_tokenize(x)
print(len(tksen))

35


In [9]:
import pandas as pd

In [10]:
#Finding the dominant topics 
import random
def format_topics_sentences(ldamodel=lda_model, corpus=termdocfreq, texts=tksen):
    # Init output
    
    dp = []
    pc = []
    tk = []
    
    
    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        #print(row_list)
        row = row_list[0] if ldamodel.per_word_topics else row_list 
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                x = random.randint(0, 9)
                topic_num=(topic_num+x)%10
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                #print(topic_keywords + " "+str(topic_num)+" "+str(prop_topic))
                #sent_topics_df = pd.concat([sent_topics_df, pd.Series([int(topic_num), round(prop_topic,4), topic_keywords])], ignore_index=True)
                #df = pd.DataFrame({'Dominant_Topic': topic_num,'Perc_Contribution':prop_topic,'Topic_Keywords':topic_keywords})
                #sent_topics_df = pd.concat([sent_topics_df, df], ignore_index=True)
                dp.append(topic_num)
                pc.append(prop_topic)
                tk.append(topic_keywords)
            else:
                break
   
    sent_topics_df = pd.DataFrame({'Dominant_Topic' : dp, 'Perc_contribution':pc, 'topic_keywords':tk })
    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)
    

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=termdocfreq, texts = tksen)
df_len=len(df_topic_sents_keywords.index)
df_topic_sents_keywords=df_topic_sents_keywords.drop(df_topic_sents_keywords.index[len(tksen):])

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_len=len(df_topic_sents_keywords.index)
print(df_len)
df_dominant_topic.head(df_len)

35


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,8,0.549954,"language, word, learning, phrase, learn, readi...",Possibly the easiest thing you can do is watch...
1,1,5,0.699966,"language, word, learning, learn, help, time, n...","Try to avoid subtitles, so you donâ€™t develop..."
2,2,2,0.774973,"word, language, learning, learn, help, importa...","To make things easier, try to watch shows or m..."
3,3,3,0.819981,"language, learning, learn, phrase, important, ...","As your language level progresses, you can mov..."
4,4,9,0.849985,"word, language, learning, learn, help, reading...","When you read out loud, rather than silently, ..."
5,5,6,0.871416,"learn, language, learning, word, thing, phrase...","Rather than trying to translate each phrase, f..."
6,6,2,0.887489,"word, language, learning, learn, help, importa...",Mastering everyday conversation will let you h...
7,7,3,0.899989,"language, learning, learn, phrase, important, ...",Itâ€™s more important to master basic phrases ...
8,8,9,0.909990,"word, language, learning, learn, help, reading...",Mastering basic vocabulary is probably one of ...
9,9,1,0.918170,"language, thing, watch, show, movie, reading, ...","Even if you can't understand whole sentences, ..."


In [11]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head(len(tksen))


,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0,0.971872,"word, language, learning, learn, thing, help, ...",If you want to learn a new language ''quickly'...
1,1,0.962496,"language, thing, watch, show, movie, reading, ...","Too often, people spend all of their time stud..."
2,2,0.967854,"word, language, learning, learn, help, importa...","In addition, looking up the word and using it ..."
3,3,0.965381,"language, learning, learn, phrase, important, ...",Carrying a dictionary will allow you to find t...
4,4,0.973527,"word, learning, language, learn, help, reading...",You might find yourself in situations where yo...
5,5,0.974997,"language, word, learning, learn, help, time, n...",If you can visit and spend some time in a coun...
6,6,0.975673,"learn, language, learning, word, thing, phrase...",Learning a language within the context of cult...
7,7,0.974283,"word, language, thing, learn, learning, phrase...",Thatâ€™s all a normal part of the language-lea...
8,8,0.972724,"language, word, learning, phrase, learn, readi...",When you're learning a new language youâ€™ll g...
9,9,0.970965,"word, language, learning, learn, help, reading...","But when they say five years, they probably me..."


In [12]:
sent_topics_sorteddf_mallet.to_csv(r"C:\Users\kanis\Major-Project\PreClusteringData\mrdfet.csv")
#saving the most representative document for each topic 

In [13]:
df_dominant_topic.to_csv(r"C:\Users\kanis\Major-Project\PreClusteringData\dt.csv")
#saving the topic wise information for each sentence or document

In [14]:
import csv 


#making the list of clusters with their sentences classfied accordingly 
listofclusters = [None] * num_topics
for i in range(num_topics):
    listofclusters[i] = []

with open(r'C:\Users\kanis\Major-Project\PreClusteringData\dt.csv', 'r',encoding="utf8") as csvfile :
    datareader = csv.reader(csvfile)
    for row in datareader:
        if row[0] == '':
            continue
        listofclusters[int(row[2])].append(row[5])
     
    
#loop for reprsentation
i = 0
for item in listofclusters :
    #print("topic cluster no. "+str(i))
    #for sentence in item:
        #print(sentence)
    #print()
    i = i + 1
    

["Even if you can't understand whole sentences, the ability to pick out keywords can help you to understand the general meaning of a speech or text.", 'The specifics of grammar will come later.', 'In turn, better reading and pronunciation can help you memorize words with greater ease.', "Too often, people spend all of their time studying grammar and memorizing lists of words instead of actually going out there and putting what they've learned into practice."]


In [15]:
import re 
import nltk 
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx

In [16]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()
len(word_embeddings)

400000

In [32]:
listofsentence_vectors = []
sentence_vectors = []
for sentences in listofclusters:
    for i in sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
    listofsentence_vectors.append(sentence_vectors)


In [18]:
import sklearn

In [36]:
# similarity matrix
from sklearn.metrics.pairwise import cosine_similarity
k = 0

listofsimmatrix = []
for sentences in listofclusters:
    sim_mat = np.zeros([len(sentences), len(sentences)])
    #We will use Cosine Similarity to compute the similarity between a pair of sentences.


#And initialize the matrix with cosine similarity scores.
    
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(listofsentence_vectors[k][i].reshape(1,100), listofsentence_vectors[k][j].reshape(1,100))[0,0]
    k=k+1
    listofsimmatrix.append(sim_mat)
#print(len(listofsimmatrix))

10


In [58]:
listofsubsum = [None] * num_topics
for i in range(num_topics):
    listofsubsum[i] = []

k = 0
for sim_mat in listofsimmatrix:
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(listofclusters[k])), reverse=True)
    for i in range(int(len(ranked_sentences)/3)):
        listofsubsum[k].append(ranked_sentences[i][1])
    k=k+1
#print(listofsubsum)
#print()
#print(listofclusters)


#generating the final list
finallist = []
for lis in listofsubsum :
    for sent in lis :
        if len(sent)>0:
            finallist.append(sent)
#print(finallist)

In [64]:
#creating dictionary to arrange summaryin chronological order
dictofsent = {}
k = 0
for sent in tksen :
    dictofsent[sent] = k
    k=k+1
summarytosave=sorted(finallist, key=dictofsent.get)
#print(summarytosave)

sample = ""
for sent in summarytosave:
    sample += sent 
    sample += ' '


Mastering everyday conversation will let you hit the ground running. Itâ€™s more important to master basic phrases youâ€™ll use frequently than it is to start off by learning the alphabet or the perfect sentence structure. Mastering basic vocabulary is probably one of the most important things you can do when learning a new language. There's no point in memorizing hundreds of words and phrases if you pronounce them so oddly that they can't be understood. The reason why most people can't remember most of the language they spent years learning in school is that school curricula tend to focus a huge amount of time on learning grammar and very little time on speech (or "comprehensible output"). This is pretty much backwardsâ€”if you want to learn a language quickly, you should learn how to converse first. The specifics of grammar will come later. Plus, it is better for you to be sounding the words out rather than looking at the romanization for the words. Speaking with a real, live person 